# TaskWeaver

This notebook aims to condence all requirements needed for coding up a TaskWeaver project.  <br>
https://export.arxiv.org/pdf/2311.17541 <br>

#### Step 1:
- Firstly, you need to make a project folder. this is a prerequisite to running TaskWeaver. See [starter_project]("../_starter_projects/") <br>

#### Step 2:
- Then we need to make a session:
    > TaskWeaver supports a basic session management to keep different users' <br>
    > data separate. The code execution is separated into different processes <br>
    > in order not to interfere with each other.

#### Step 3:
- We also need to enable modules that the code interpreter can use and code verification.
    > Code verification - TaskWeaver is designed to verify the generated code before execution. <br>
    > It can detect potential issues in the generated code and provide suggestions to fix them.
    
#### Step 4:
- TaskWeaver has restricted library use to mitigate against unforeseen consequences of running code, thus you need to call out the permitted libraries expicitly.
- Add this to the `taskweaver_config.json`:
    ```
    "code_verification.allowed_modules": ["pandas", "matplotlib", "numpy", "sklearn", "scipy", "seaborn", "datetime", "typing"],
    "code_verification.code_verification_on": true,
    ```

In [1]:
import json
from typing import Any, Optional

import pprint as pp
from taskweaver.app.app import TaskWeaverApp

# Create a session
# would prefer if the expected folder structure was autogenerated
app_dir = "./_starter_projects/"

### Configuration

In [ ]:
# Here is where the configurations from taskweaver_config.json are stored
# https://github.com/microsoft/TaskWeaver/blob/main/docs/configuration.md 
# Categories of keys within the config
# - session_manager
# - llm
# - workspace
# - logging
# - session
# - planner
# - plugin
# - round_compressor
# - code_generator
# - embedding_model
# - code_verification
# - code_interpreter
### session.config.src.config


### Simple test message

In [ ]:
app = TaskWeaverApp(app_dir=app_dir)
session = app.get_session()

# Send a message to the session
user_query = "hello, what can you do?"
response_round = session.send_message(
                            user_query,
                            event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}")
                          )

In [ ]:
response_round.to_dict()['post_list'][-1]['message']

In [ ]:
# Heres what that looks like in JSON
pp.pprint(response_round.to_dict())

### The classic stock market agent benchmark

protip: make sure `llm.response_format` is `text` because when working with `json` i found it crashed when using the code interpreter.

> 💡 Up to 11/30/2023, the json_object and text options of `llm.response_format` is only supported by the OpenAI models later than `1106`. If you are using an older version of OpenAI model, you need to set the llm.response_format to null.

In [ ]:
user_query = "Write code to plot the price of tesla stock YTD and save it to the current working directory"
response_round = session.send_message(user_query,
                                      event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}"))

![](./_starter_projects/tesla_stock_price_ytd.png)

![](../_starter_projects/workspace/sessions/20231212-200217-2ba37e82/cwd/tesla_stock_price_ytd.png)

### Plugins

A plugin is the equivalent to a function call in Autogen, but it handles a lot of the orchestration for you. All you need to do is put the python file and yaml file in the `plugins` folder.
Make sure you update `taskweaver_config.json` with the following:
```json
"enable_auto_plugin_selection": "true"
```

In [ ]:
# session.config.src.config

In [2]:
app = TaskWeaverApp(app_dir=app_dir)
session = app.get_session()

user_query = "Perform analysis over the dataframe"
response_round = session.send_message(user_query,
                                      event_handler=lambda _type, _msg: print(f"{_type}:\n{_msg}"))

BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [3]:
! python --version

Python 3.10.12
